In [136]:
import face_recognition as fr
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import glob
import numpy as np
import re
import pandas as pd

In [137]:
path = "data/CK+/"
data  = listdir(path)
ignore = ["morralla",".DS_Store"]
imgs = []
state = []

In [138]:
for item in data:
    if item not in ignore:
        imgs.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
        state.extend([item for p in listdir(f"{path}{item}")])

In [139]:
state[:5], imgs[:5]

(['surprise', 'surprise', 'surprise', 'surprise', 'surprise'],
 ['data/CK+/surprise/S052_001_00000015.png',
  'data/CK+/surprise/S050_002_00000018.png',
  'data/CK+/surprise/S086_001_00000019.png',
  'data/CK+/surprise/S037_001_00000020.png',
  'data/CK+/surprise/S034_001_00000029.png'])

In [140]:
imgs = [Image.open(p) for p in imgs]

In [141]:
imgs

[<PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060FC70>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C50695AF0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C50695F70>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C506958B0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060FEE0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060F310>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060F400>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060FC10>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060F2E0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060FCA0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060FA60>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F4C5060F040>,
 <PIL.PngImagePlugin.PngImageFile image 

In [142]:
imgs_array = []
HEIGHT, WIDTH =200,200
for f in imgs:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array.append(np.array(img))

In [143]:
#imgs_array = [el/255 for el in imgs_array]


In [144]:
imgs_array = np.array(imgs_array)

In [145]:
imgs_array.shape[1:]

(200, 200, 3)

In [146]:
categories = list(set(state))

In [147]:
y=pd.DataFrame(state)
y_dummies = pd.get_dummies(y)

In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs_array,y_dummies)

In [155]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from tensorflow.keras.optimizers import Adam
import keras

In [156]:
model = Sequential()
#model.add(Flatten())
model.add(Conv2D(filters=64, kernel_size=2, activation='relu', input_shape=(imgs_array.shape[1:])))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(filters=256, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(filters=512, kernel_size=2, activation='relu', input_shape=(imgs_array.shape[1:])))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [157]:
from keras.models import Sequential, load_model, model_from_json
from keras import callbacks, optimizers
import tensorflow as tf

symbol = 'complet_faces'
h5 = symbol + '_best_model' + '.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                       monitor='loss',
                                       verbose=0,
                                       save_best_only=True,
                                       #save_weights_only=True,
                                       mode='auto',
                                       period=1)
callback = [checkpoint]
json = symbol + '_best_model' + '.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)

modelo = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 5000,callbacks = callback,validation_split = 0.1)

Epoch 1/5000
20/20 [==============================] - 4s 121ms/step - loss: 2.5112 - accuracy: 0.1820 - val_loss: 8.0246 - val_accuracy: 0.0290
Epoch 2/5000
20/20 [==============================] - 2s 101ms/step - loss: 2.1583 - accuracy: 0.2464 - val_loss: 2.3767 - val_accuracy: 0.4058
Epoch 3/5000
20/20 [==============================] - 2s 98ms/step - loss: 1.9069 - accuracy: 0.3655 - val_loss: 2.3105 - val_accuracy: 0.2029
Epoch 4/5000
20/20 [==============================] - 2s 76ms/step - loss: 1.7862 - accuracy: 0.4203 - val_loss: 2.3493 - val_accuracy: 0.3188
Epoch 5/5000
20/20 [==============================] - 2s 77ms/step - loss: 1.6067 - accuracy: 0.5217 - val_loss: 3.1075 - val_accuracy: 0.0580
Epoch 6/5000
20/20 [==============================] - 2s 99ms/step - loss: 1.5405 - accuracy: 0.5443 - val_loss: 1.8210 - val_accuracy: 0.3913
Epoch 7/5000
20/20 [==============================] - 2s 85ms/step - loss: 1.4958 - accuracy: 0.5845 - val_loss: 1.5281 - val_accuracy: 0.57

20/20 [==============================] - 1s 65ms/step - loss: 0.7185 - accuracy: 0.7536 - val_loss: 1.2867 - val_accuracy: 0.6377
Epoch 58/5000
20/20 [==============================] - 1s 65ms/step - loss: 0.8028 - accuracy: 0.7440 - val_loss: 1.3116 - val_accuracy: 0.6957
Epoch 59/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.7127 - accuracy: 0.7520 - val_loss: 1.1695 - val_accuracy: 0.6667
Epoch 60/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.7196 - accuracy: 0.7649 - val_loss: 1.2142 - val_accuracy: 0.6377
Epoch 61/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.7347 - accuracy: 0.7681 - val_loss: 0.9467 - val_accuracy: 0.6812
Epoch 62/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.6912 - accuracy: 0.7585 - val_loss: 1.1836 - val_accuracy: 0.6667
Epoch 63/5000
20/20 [==============================] - 1s 65ms/step - loss: 0.7257 - accuracy: 0.7520 - val_loss: 1.3325 - val_accuracy: 0.6667
Epoch 

Epoch 114/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.2244 - accuracy: 0.9227 - val_loss: 1.5501 - val_accuracy: 0.6522
Epoch 115/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.1760 - accuracy: 0.9324 - val_loss: 1.5320 - val_accuracy: 0.6377
Epoch 116/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.1877 - accuracy: 0.9420 - val_loss: 1.4934 - val_accuracy: 0.6377
Epoch 117/5000
20/20 [==============================] - 1s 75ms/step - loss: 0.1393 - accuracy: 0.9565 - val_loss: 1.2417 - val_accuracy: 0.7246
Epoch 118/5000
20/20 [==============================] - 1s 63ms/step - loss: 0.2421 - accuracy: 0.9243 - val_loss: 1.6651 - val_accuracy: 0.6377
Epoch 119/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.2288 - accuracy: 0.9227 - val_loss: 1.3231 - val_accuracy: 0.6522
Epoch 120/5000
20/20 [==============================] - 1s 63ms/step - loss: 0.1696 - accuracy: 0.9420 - val_loss: 1.4024 - val_ac

20/20 [==============================] - 1s 66ms/step - loss: 0.0493 - accuracy: 0.9823 - val_loss: 1.3345 - val_accuracy: 0.6667
Epoch 171/5000
20/20 [==============================] - 1s 65ms/step - loss: 0.0281 - accuracy: 0.9887 - val_loss: 1.3988 - val_accuracy: 0.6957
Epoch 172/5000
20/20 [==============================] - 1s 65ms/step - loss: 0.0246 - accuracy: 0.9903 - val_loss: 1.3155 - val_accuracy: 0.7101
Epoch 173/5000
20/20 [==============================] - 1s 67ms/step - loss: 0.0412 - accuracy: 0.9903 - val_loss: 1.4503 - val_accuracy: 0.7101
Epoch 174/5000
20/20 [==============================] - 1s 67ms/step - loss: 0.0603 - accuracy: 0.9855 - val_loss: 1.5408 - val_accuracy: 0.7246
Epoch 175/5000
20/20 [==============================] - 1s 66ms/step - loss: 0.0663 - accuracy: 0.9807 - val_loss: 1.5390 - val_accuracy: 0.6957
Epoch 176/5000
20/20 [==============================] - 1s 66ms/step - loss: 0.0431 - accuracy: 0.9903 - val_loss: 1.5450 - val_accuracy: 0.6667


20/20 [==============================] - 1s 64ms/step - loss: 0.0470 - accuracy: 0.9871 - val_loss: 1.5432 - val_accuracy: 0.6812
Epoch 227/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.1647 - accuracy: 0.9469 - val_loss: 1.5430 - val_accuracy: 0.7101
Epoch 228/5000
20/20 [==============================] - 1s 64ms/step - loss: 0.1570 - accuracy: 0.9501 - val_loss: 1.3974 - val_accuracy: 0.6667
Epoch 229/5000
20/20 [==============================] - 1s 63ms/step - loss: 0.1459 - accuracy: 0.9533 - val_loss: 1.7443 - val_accuracy: 0.6522
Epoch 230/5000
20/20 [==============================] - 1s 63ms/step - loss: 0.0787 - accuracy: 0.9742 - val_loss: 1.6279 - val_accuracy: 0.6957
Epoch 231/5000
20/20 [==============================] - 1s 62ms/step - loss: 0.1003 - accuracy: 0.9775 - val_loss: 1.5524 - val_accuracy: 0.6377
Epoch 232/5000
20/20 [==============================] - 1s 63ms/step - loss: 0.0365 - accuracy: 0.9887 - val_loss: 1.5503 - val_accuracy: 0.6667


20/20 [==============================] - 1s 64ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 1.6464 - val_accuracy: 0.7246
Epoch 283/5000
20/20 [==============================] - 1s 63ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 1.6299 - val_accuracy: 0.7391
Epoch 284/5000
20/20 [==============================] - 1s 65ms/step - loss: 0.0230 - accuracy: 0.9936 - val_loss: 1.7042 - val_accuracy: 0.7101
Epoch 285/5000
20/20 [==============================] - 1s 65ms/step - loss: 0.0102 - accuracy: 0.9984 - val_loss: 1.5881 - val_accuracy: 0.7246
Epoch 286/5000
20/20 [==============================] - 1s 68ms/step - loss: 0.0180 - accuracy: 0.9952 - val_loss: 2.0300 - val_accuracy: 0.6812
Epoch 287/5000
20/20 [==============================] - 1s 66ms/step - loss: 0.0114 - accuracy: 0.9968 - val_loss: 1.7510 - val_accuracy: 0.6957
Epoch 288/5000
20/20 [==============================] - 1s 66ms/step - loss: 0.0140 - accuracy: 0.9952 - val_loss: 1.7032 - val_accuracy: 0.7101


KeyboardInterrupt: 

In [158]:
path = "data/predict/"
data_pred  = listdir(path)
ignore = ["morralla",".DS_Store"]
imgs_pred = []
state_pred = []

In [159]:
for item in data_pred:
    if item not in ignore:
        imgs_pred.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
        state_pred.extend([item for p in listdir(f"{path}{item}")])
imgs_pred = [Image.open(p) for p in imgs_pred]
imgs_array_pred = []
for f in imgs_pred:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array_pred.append(np.array(img))
imgs_array_pred = np.array(imgs_array_pred)

In [160]:
prediction = model.predict(imgs_array_pred)

In [161]:
len(prediction)

20

In [162]:
prediction

array([[2.22278714e-01, 3.49985845e-02, 2.54391879e-03, 9.98574833e-05,
        1.71892476e-04, 7.35012770e-01, 3.27752461e-03, 1.61665177e-03],
       [7.18591473e-05, 9.03913926e-04, 4.85602854e-04, 2.27595195e-02,
        1.68689385e-01, 8.03282201e-01, 2.04376236e-04, 3.60313337e-03],
       [3.72109091e-04, 1.73382282e-01, 3.10280900e-02, 4.29015644e-02,
        2.72192545e-02, 7.22089171e-01, 2.58313003e-03, 4.24444093e-04],
       [2.69962737e-08, 1.61374146e-07, 2.42813103e-09, 2.74582890e-05,
        4.75845516e-01, 1.23974714e-05, 4.47558205e-06, 5.24109900e-01],
       [9.69608009e-05, 1.53672136e-05, 6.38046049e-06, 2.08122510e-05,
        8.42393129e-05, 1.57247216e-03, 6.69453584e-05, 9.98136759e-01],
       [8.44776614e-13, 6.05958728e-36, 3.14066559e-01, 1.16541765e-26,
        6.85933471e-01, 1.41675196e-08, 1.40132140e-14, 1.93515140e-20],
       [1.29834621e-06, 1.09347049e-04, 4.91172796e-05, 1.82692802e-05,
        9.99713242e-01, 7.69624094e-05, 6.22298057e-06, 2.

In [163]:
prediction.argmax(axis = -1)


array([5, 5, 5, 7, 7, 4, 4, 5, 4, 4, 3, 7, 4, 0, 2, 5, 5, 3, 4, 5])

In [164]:
cats= y_dummies.columns
cats = [x.replace("0_","") for x in cats]

In [165]:
cats

['anger',
 'contempt',
 'disgust',
 'fear',
 'happiness',
 'neutral',
 'sadness',
 'surprise']

In [166]:
states_model = []
for i in range(len(prediction)):
    states_model.append(cats[prediction[i].argmax()])
    print(state_pred[i],cats[prediction[i].argmax()])

neutral neutral
neutral neutral
neutral neutral
neutral surprise
neutral surprise
happiness happiness
happiness happiness
happiness neutral
happiness happiness
happiness happiness
happiness fear
happiness surprise
happiness happiness
happiness anger
happiness disgust
sadness neutral
sadness neutral
sadness fear
sadness happiness
sadness neutral


In [167]:
from sklearn.metrics import accuracy_score

In [168]:
accuracy_score(state_pred,states_model)

0.4